In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
print (tf.__version__)

2.1.0


In [6]:
print ("Num of GPU available {}".format(len (tf.config.experimental.list_physical_devices('GPU'))))

Num of GPU available 1


# Import the libraries
First, we are going to import all the modules that we are going to need for training our model. The Keras library already contains some datasets and MNIST is one of them. So we can easily import the dataset and start working with it. The mnist.load_data() method returns us the training data, its labels and also the testing data and its labels.

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

%conda install keras

# Split the data between train and test sets

In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [8]:
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

# Preprocess the data
The image data cannot be fed directly into the model so we need to perform some operations and process the data to make it ready for our neural network. The dimension of the training data is (60000,28,28). The CNN model will require one more dimension so we reshape the matrix to shape (60000,28,28,1).

In [9]:
x_train = x_train.reshape (x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape (x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical (y_train, 10)
y_test = keras.utils.to_categorical (y_test, 10)

x_train = x_train.astype ('float32')
x_test = x_test.astype ('float32')
x_train /= 255
x_test /= 255

# Create the model
Now we will create our CNN model in Python data science project. A CNN model generally consists of convolutional and pooling layers. It works better for data that are represented as grid structures, this is the reason why CNN works well for image classification problems. The dropout layer is used to deactivate some of the neurons and while training, it reduces over fitting of the model. We will then compile the model with the Adadelta optimizer.

In [12]:
batch_size = 128
num_classes = 10
epochs = 10

model = keras.Sequential()
model.add (layers.Conv2D (32, kernel_size = (3,3),
                   activation = 'relu',
                   input_shape = input_shape))
model.add (layers.Conv2D (64, (3,3), activation = 'relu'))
model.add (layers.MaxPooling2D (pool_size = (2,2)))
model.add (layers.Dropout (0.25))
model.add (layers.Flatten ())
model.add (layers.Dense (256, activation = 'relu'))
model.add (layers.Dropout (0.5))
model.add (layers.Dense (num_classes, activation = 'softmax'))


model.compile (loss = keras.losses.categorical_crossentropy, 
               optimizer  = keras.optimizers.Adadelta(),
               metrics = ['accuracy'])

# Train the model

In [13]:
hist = model.fit (x_train, y_train, 
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = 1,
                  validation_data = (x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 10s 165us/sample - loss: 2.2766 - accuracy: 0.1433 - val_loss: 2.2330 - val_accuracy: 0.3482
Epoch 2/10
60000/60000 [==============================] - 7s 122us/sample - loss: 2.2017 - accuracy: 0.2807 - val_loss: 2.1398 - val_accuracy: 0.5288
Epoch 3/10
60000/60000 [==============================] - 7s 119us/sample - loss: 2.1027 - accuracy: 0.4005 - val_loss: 2.0095 - val_accuracy: 0.6369
Epoch 4/10
60000/60000 [==============================] - 7s 119us/sample - loss: 1.9630 - accuracy: 0.5000 - val_loss: 1.8273 - val_accuracy: 0.7157
Epoch 5/10
60000/60000 [==============================] - 7s 124us/sample - loss: 1.7731 - accuracy: 0.5827 - val_loss: 1.5900 - val_accuracy: 0.7607
Epoch 6/10
60000/60000 [==============================] - 7s 116us/sample - loss: 1.5479 - accuracy: 0.6358 - val_loss: 1.3280 - val_accuracy: 0.7868
Epoch 7/10
60000/60000 [=========================

In [23]:
# Save the model
import joblib
joblib.dump (model, 'digit_recognizer.joblib')

TypeError: can't pickle _thread.RLock objects

# Evaluate the model

In [15]:
score = model.evaluate (x_test, y_test, verbose = 0)

In [16]:
score

[0.6908139512062073, 0.8391]

In [17]:
print ('Test loss: {}'.format (score [0]))
print ("Test accuracy: {}".format (score [1]))

Test loss: 0.6908139512062073
Test accuracy: 0.8391000032424927


# Create a GUI to predict digits
**This is recommended to have it in a separate file**

Now for the GUI, we have created a new file in which we build an interactive window to draw digits on canvas and with a button, we can recognize the digit. The Tkinter library comes in the Python standard library. We have created a function predict_digit() that takes the image as input and then uses the trained model to predict the digit.

Then we create the App class which is responsible for building the GUI for our app. We create a canvas where we can draw by capturing the mouse event and with a 

In [18]:
import joblib
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [22]:
# load the model
model = joblib.load ('digit_recognizer.joblib')

def predict_digit (img):
    """
    Takes the image input and uses the trained model to predict the output

    params:
        input: takes the image input from a canvas window
        output: returns the digit 
    """

    # resize image to 28x28 pixels
    img = img.resize ((28, 28))

    #convert rgb to grayscale 
    img = img.convert ('L')
    img = np.array (img)

    # reshaping to support our model input and normalizing
    img = img.reshape (1, 28, 28, 1)
    img = img/255.0

    # predicting the class
    result = model.predict ([img])[0]
    return np.argmax (result), max (result)


class App (tk.Tk):
    def __init__ (self):
        tk.Tk.__init__(self)
        
        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas (self, width = 300, height = 300, 
                                 bg = 'white', cursor = 'cross')
        self.label = tk.Label (self, text = 'Thinking..',
                               font = ("Helvetica", 48))
        self.classify_btn = tk.Button (self, text = "Recognize",
                                       command = self.classify_handwriting)
        self.button_clear = tk.Button (self, text = "Clear", command  = self.clear_all)
        
        
        # Grid Structure 
        self.canvas.grid (row = 0, column = 0, pady = 2, sticky = "W")
        self.label.grid (row = 0, column = 0, pady = 2, padx = 2)
        self.classify_btn.grid (row = 1, column = 1, pady = 2, padx = 2)
        self.button_clear.grid (row = 1, column = 0, pady = 2)
        
        self.canvas.bind ("<B1-Motion>", self.draw_lines)
        
    def clear_all (self):
        self.canvas.delete ("all")
        
    def classify_handwriting (self):
        HWND = self.canvas.winfo_id () # get the handle of the canvas
        rect = win32gui.GetWindowRect (HWND)  # get the coordinate of the canvas
#         a,b,c,d = rect
#         rect = (a+4, b+4, c+4, d+4)
        img = ImageGrab.grab (rect)
        
        digit, acc = predict_digit (img)
        self.label.configure (text = str(digit) + ", "+ str(int (acc*100))+ "%")
        
    def draw_lines (self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval (self.x-r, self.y-r, 
                                 self.x + r, self.y + r, 
                                 fill='black')

App().mainloop()

EOFError: 